# Training Data Preparation

## Setup

In [93]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('../..')
import data
from data.labels_postprocessing import process
from data.dataframe_preparation import get_counts_per_page, get_keywords_from_file, get_text_from_page, get_count_matrix
from data.preprocessing import DocumentPreprocessor

### Load labelling files

The 100 randomly selected reports were labelled by two labellers:

- Labeller A: Main labeller, labelled the first 78 and last 5 reports
- Labeller B: Labelled the first 15 reports for Inter-Coder reliability checks (see Notebook) and also labelled 17 of the last few

In [94]:
df_header_a = pd.read_csv("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Training_DF.csv") 
df_labels_a = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_DF.pkl")
df_header_b = pd.read_csv("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Training_TS.csv")
df_labels_b = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_TS.pkl")

In [95]:
# Assign coder ids
df_labels_a['coder'] = "DF"
df_labels_b['coder'] = "TS"

# Remove all labels (of coder B) from reports selected for ICR checks
filtered_df_labels_b = df_labels_b.iloc[69:].copy()

# Postprocess, e.g. extract unstructerd comment field
df_labels_a = process(df_labels_a)
filtered_df_labels_b = process(filtered_df_labels_b)

# Combine dataframes
df_labels_positive = pd.concat([df_labels_a, filtered_df_labels_b], ignore_index=True)

In [96]:
# Show all labels without categories: 
df_labels_positive.query("cro.isnull() | cro == ''")

,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined
55,nl_asml_hldg-AR_2013,NaN,NaN,44,1,True,"negative,neg co2 example",The�number�of�lithography�systems�we�are�able�...,DF,False,False,False,NaN,NaN,None
91,es_telefonica-AR_2016,,,166,9,True,,We have analysed our vulnerability \nto climat...,DF,False,False,False,NaN,NaN,None
133,es_telefonica-AR_2018,NaN,NaN,109,2,True,"vague,special:contains everything",For physical risks we analyse the probability ...,DF,False,True,False,NaN,NaN,None
286,fr_axa-AR_2012,NaN,NaN,387,3,True,interesting example that is not directly an OP,(see above sections). Indeed insurers are well...,DF,False,False,False,NaN,NaN,None
313,fr_axa-AR_2017,NaN,NaN,385,4,True,not directly a CRO but interesting example on ...,AXA is reviewing a framework that models trans...,DF,False,False,False,NaN,NaN,None
314,fr_axa-AR_2017,NaN,NaN,385,5,True,not directly a CRO but interesting example on ...,These results can be translated into a forward...,DF,False,False,False,NaN,NaN,None
315,fr_axa-AR_2017,NaN,NaN,385,7,True,not directly a CRO but interesting example on ...,"In addition to the above “transition” risks, c...",DF,False,False,False,NaN,NaN,None
316,fr_axa-AR_2017,NaN,NaN,385,8,True,not directly a CRO but interesting example on ...,Our physical risk assessment uses “NatCat” mod...,DF,False,False,False,NaN,NaN,None
317,fr_axa-AR_2017,NaN,NaN,385,9,True,not directly a CRO but interesting example on ...,"Our results, which are based on an exploratory...",DF,False,False,False,NaN,NaN,None
686,gb_glencore_plc-AR_2017,NaN,NaN,15,5,NaN,"No direct risk, but CC relevant nevertheless",• As a major producer and consumer of fossil \...,DF,False,False,False,NaN,NaN,None


In [97]:
df_labels_positive.loc[91]

report_id                                            es_telefonica-AR_2016
cro                                                                       
cro_sub_type                                                              
page                                                                   166
paragraph_no                                                             9
label                                                                 True
comment                                                                   
text                     We have analysed our vulnerability \nto climat...
coder                                                                   DF
indirect                                                             False
vague                                                                False
past                                                                 False
keyword                                                                NaN
span_id                  

In [98]:
# Tricky one, however the only real risk they report is at the end.
df_labels_positive.loc[133, "cro"] = "TR"
df_labels_positive.loc[133, "cro_sub_type"] = "MARKET"

# This can be labelled as a transition risk/
df_labels_positive.loc[713, "cro"] = "TR"
df_labels_positive.loc[713, "cro_sub_type"] = "POLICY"

# Based on the later revised codebook, even very "indirect"/vague mentionings of physical risks should be labelled as such
df_labels_positive.loc[716, "cro"] = "PR"
df_labels_positive.loc[716, "cro_sub_type"] = "ACUTE"
df_labels_positive.loc[716, "comment"] = "indirect,vague"

# Could also be an opportunity?
df_labels_positive.loc[717, "cro"] = "TR"
df_labels_positive.loc[717, "cro_sub_type"] = "MARKET"

# Not really a risk disclosure?
df_labels_positive = df_labels_positive.drop([91])

In [99]:
# Fix labels that do not have a sub-category.
df_labels_positive.query("(cro_sub_type.isnull() | cro_sub_type == '') and cro")

,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined
1021,gb_rio_tinto-AR_2010,PR,NaN,29,10,True,NaN,"Mining, smelting and refining operations are v...",TS,False,False,False,NaN,NaN,NaN
1024,gb_rio_tinto-AR_2010,OP,NaN,33,5,True,vague,The minerals and metals produced at our operat...,TS,False,True,False,NaN,NaN,None
1025,gb_rio_tinto-AR_2010,OP,NaN,36,17,True,vague,Greenhouse gas emissions\nWe accept the need f...,TS,False,True,False,NaN,NaN,None
1039,gb_bp-AR_2017,OP,NaN,14,6,True,vague,"Through new technologies, energy will \nbe pro...",TS,False,True,False,NaN,NaN,None
1049,gb_bp-AR_2017,TR,NaN,58,1,True,vague,We identify high priority risks for particular...,TS,False,True,False,NaN,NaN,None
1078,gb_bp-AR_2010,PR,NaN,75,2,True,NaN,Adaptation to climate change impacts\nFor seve...,TS,False,False,False,NaN,NaN,NaN
1079,gb_bp-AR_2010,PR,NaN,75,3,True,NaN,For larger projects where climate impacts are ...,TS,False,False,False,NaN,NaN,NaN
1089,gb_royal_dutch_shell_a-AR_2011,PR,NaN,16,3,True,NaN,The nature of our operations exposes us to a w...,TS,False,False,False,NaN,NaN,NaN
1131,fr_total-AR_2016,PR,NaN,69,10,True,NaN,The physical effects of climate change may adv...,TS,False,False,False,NaN,NaN,NaN
1140,fr_total-AR_2016,PR,,164,15,True,,The Group ensures that it assesses the vulnera...,TS,False,False,False,NaN,NaN,


In [100]:
tmp_id = 1139
print(df_labels_positive.loc[tmp_id], df_labels_positive.text[tmp_id])

report_id                                                 fr_total-AR_2016
cro                                                                     TR
cro_sub_type                                                        POLICY
page                                                                   162
paragraph_no                                                            10
label                                                                 True
comment                                                                NaN
text                     Furthermore, the Group ensures sustainability ...
coder                                                                   TS
indirect                                                             False
vague                                                                False
past                                                                 False
keyword                                                                NaN
span_id                  

In [101]:
# Floods, droughts
df_labels_positive.loc[1021, "cro"] = "PR"
df_labels_positive.loc[1021, "cro_sub_type"] = "ACUTE"

# A very greenwashy example...
df_labels_positive.loc[1024, "cro_sub_type"] = "PRODUCTS"

# This is actually not an opportunity so lets remove
df_labels_positive.loc[1025, "cro"] = None

# Products..
df_labels_positive.loc[1039, "cro_sub_type"] = "PRODUCTS"

# Remove since its not explicitly a risk..
df_labels_positive.loc[1049, "cro"] = None

# Not a risk
df_labels_positive.loc[1078, "cro"] = None

# Not a explicit risk
df_labels_positive.loc[1079, "cro"] = None

# Very far stretched PR...
df_labels_positive.loc[1089, "cro_sub_type"] = "ACUTE"

# Could be both ACUTE and CHRONIC....
df_labels_positive.loc[1131, "cro_sub_type"] = "ACUTE"
new_row = df_labels_positive.loc[1131].copy()
new_row.cro_sub_type = "CHRON"
df_labels_positive = df_labels_positive.append(new_row, ignore_index=True)

# Boilerplate examples
df_labels_positive.loc[1144, "cro_sub_type"] = "ACUTE"
new_row = df_labels_positive.loc[1144].copy()
new_row.cro_sub_type = "CHRON"
df_labels_positive = df_labels_positive.append(new_row, ignore_index=True)

df_labels_positive.loc[1145, "cro_sub_type"] = "REPUT"
new_row = df_labels_positive.loc[1145].copy()
new_row.cro_sub_type = "MARKET"
df_labels_positive = df_labels_positive.append(new_row, ignore_index=True)
new_row.cro_sub_type = "POLICY"
df_labels_positive = df_labels_positive.append(new_row, ignore_index=True)

# More of a risk adaption
df_labels_positive.loc[1139, "cro_sub_type"] = "ACUTE"
new_row = df_labels_positive.loc[1139].copy()
new_row.cro_sub_type = "CHRON"
df_labels_positive = df_labels_positive.append(new_row, ignore_index=True)

In [ ]:
# Show and remove duplicates
dupl = df.groupby(["report_id", "cro", "cro_sub_type", "page", "paragraph_no"]).size()
print(dupl[dupl > 1])

df_labels_positive = df_labels_positive.drop(df_labels_positive.query("report_id == 'de_daimler-AR_2003' and page == 75")[1])
# TODO: df_labels_positive = df_labels_positive.drop([XXXX])

In [102]:
df_labels_positive

,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined
0,de_sap-AR_2011,OP,PRODUCTS,83,2,True,"cro_id:sustain_management,indirect",With the continuing threat of climate change a...,DF,True,False,False,NaN,sustain_management,PRODUCTS
1,de_sap-AR_2011,OP,PRODUCTS,83,3,True,"cro_id:sustain_management,indirect",Vision and Mission\nOur vision is to help the ...,DF,True,False,False,NaN,sustain_management,PRODUCTS
2,de_sap-AR_2011,OP,PRODUCTS,107,3,True,"indirect,vague","– SAP Smart Meter Analytics software, launched...",DF,True,True,False,NaN,NaN,PRODUCTS
3,de_sap-AR_2011,OP,RESILI,118,2,True,NaN,"eneRgY And emIssIons\nServing our customers, p...",DF,False,False,False,NaN,NaN,RESILIENCE
4,de_sap-AR_2011,OP,RESILI,119,0,True,"indirect,vague",– Data center energy: We focus on making data ...,DF,True,True,False,NaN,NaN,RESILIENCE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1182,fr_total-AR_2016,PR,CHRON,69,10,True,NaN,The physical effects of climate change may adv...,TS,False,False,False,NaN,NaN,NaN
1183,it_eni-AR_2018,TR,CHRON,26,6,True,keyword:transition,Climate change referred to the possibility of ...,TS,False,False,False,transition,NaN,None
1184,it_eni-AR_2018,TR,MARKET,93,1,True,NaN,- Eni is facing strong competitive pressure in...,TS,False,False,False,NaN,NaN,MARKET
1185,it_eni-AR_2018,TR,POLICY,93,1,True,NaN,- Eni is facing strong competitive pressure in...,TS,False,False,False,NaN,NaN,MARKET


## Fix for reports that were labelled with an outdated version of the preprocessor

Some reports were labelled (at the beginning) with an outdated version of the preprocessor, where for layouts the paragraph rules did not break them up properly.
Some manual fixes for labels are applied here. Would have to get done in a more robust fashion...

In [103]:
parsed_reports_folder = "/Users/david/Projects/fin-disclosures-nlp/input_files/annual_reports"

def get_paragraphs(company_id, report_year, page_no, paragraph_no):
    rows = df_labels_positive.query(f"report_id == '{company_id}-AR_{report_year}' & page == {page_no} & paragraph_no == {paragraph_no}")
    doc = get_text_from_page(os.path.join(parsed_reports_folder, company_id, f"AR_{report_year}", f"AR_{report_year}.yml"), page_no)
    processed_doc = DocumentPreprocessor(doc).process()
    paragraphs = processed_doc.split('\n\n')
    return rows, paragraphs

# No correction possible: rows = df_labels_positive.query("report_id == 'gb_bp-AR_2017' & page == 268 & paragraph_no == 5")
# No correction possible: rows = df_labels_positive.query("report_id == 'gb_diageo-AR_2016' & page == 158 & paragraph_no == 1")


In [104]:
# No correction possible/necessary:
rows, paragraphs = get_paragraphs('gb_bp', 2017, 268, 5)
rows, paragraphs = get_paragraphs('gb_diageo', 2016, 158, 1)

# Correction possible
rows, paragraphs = get_paragraphs('it_eni', 2018, 113, 3)
print("# of paragraphs: ", len(paragraphs))
rows

# of paragraphs:  11


,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined
1159,it_eni-AR_2018,OP,MARKETS,113,3,True,keywords:strategic,Main risks and opportunities\nClimate change i...,TS,False,False,False,NaN,NaN,PRODUCTS
1160,it_eni-AR_2018,TR,POLICY,113,3,True,keyword:regulatory,Main risks and opportunities\nClimate change i...,TS,False,False,False,regulatory,NaN,POLICY
1161,it_eni-AR_2018,TR,TECH,113,3,True,keyword:breakthrough,Main risks and opportunities\nClimate change i...,TS,False,False,False,breakthrough,NaN,MARKET
1162,it_eni-AR_2018,TR,REPUT,113,3,True,keyword:reputation,Main risks and opportunities\nClimate change i...,TS,False,False,False,reputation,NaN,REPUTATION
1163,it_eni-AR_2018,PR,ACUTE,113,3,True,keyword:extreme,Main risks and opportunities\nClimate change i...,TS,False,False,False,extreme,NaN,ACUTE
1164,it_eni-AR_2018,PR,CHRON,113,3,True,keyword:chronic,Main risks and opportunities\nClimate change i...,TS,False,False,False,chronic,NaN,CHRON


In [72]:
print(paragraphs[3])

Main risks and opportunities
Climate change is analysed, evaluated and managed by 
considering energy transition aspects (market scenario, regulatory 
and technological evolution, reputational issues) and physical 
phenomena. The analysis is carried out using an integrated and 
cross-cutting approach which involves specialist departments and 
business lines and considers the related risks and opportunities. 
The main findings are shown below.
Market scenario. In the IEA Sustainable Development Scenario5 
(WEO 2018), taken as a reference to assess the risks of the energy 
transition, fossil fuels are expected to continue to play a central 
role in the energy mix (Oil & Gas equal to 48% of the mix in 2040), 
although in this scenario the global energy demand by 2040 is 
expected to fall slightly. Natural gas, which grows also in the SDS 
scenario, represents an opportunity for strategic repositioning for 
energy companies, due to its lower carbon intensity, the possibility 
of integratio

In [105]:
# Eni long page text fixes
df_labels_positive.loc[1160, "paragraph_no"] = 5
df_labels_positive.loc[1161, "paragraph_no"] = 6
df_labels_positive.loc[1162, "paragraph_no"] = 7
df_labels_positive.loc[1163, "paragraph_no"] = 8
df_labels_positive.loc[1164, "paragraph_no"] = 9

In [106]:
# Run again to make sure we have all changes
df_labels_positive = process(df_labels_positive)

In [107]:
# Filter out rows that have NO cro category and assign as "good" negative example
df_labels_negative = df_labels_positive.query("cro.isnull()").copy()
df_labels_negative['neg_type'] = "manual"
len(df_labels_negative)

15

In [108]:
df_labels_positive[df_labels_positive.comment.str.find("water") > 0]
df_labels_positive[df_labels_positive.comment.str.find("recycling") > 0]

,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined
81,es_telefonica-AR_2016,OP,EFFI,159,1,True,keyword:recycling,Circular economy_\nRethinking the model \nThe ...,DF,False,False,False,recycling,NaN,RESILIENCE
197,de_bayer-AR_2016,OP,EFFI,134,2,True,cro_id:site_recycling,Material-based recycling is important in Crop ...,DF,False,False,False,NaN,site_recycling,RESILIENCE
198,de_bayer-AR_2016,OP,EFFI,134,3,True,cro_id:site_recycling,"Crop Science’s global Environment Policy, all ...",DF,False,False,False,NaN,site_recycling,RESILIENCE
301,fr_axa-AR_2017,OP,EFFI,377,9,True,"water,recycling","In addition, the Group has also set two enviro...",DF,False,False,False,NaN,NaN,RESILIENCE


## Generation of negative samples

Take all paragraphs from bigram hits that were NOT labelled as such.

In [19]:
# Set temporary tracking lists
hot_negative_labels = []
weak_negative_labels = []

# Loop through each report in df_header_a that was actually labelled.
labelled_reports = df_header_a.query("should_label & is_labelled & not label_bug")

parsed_reports_folder = "/Users/david/Projects/fin-disclosures-nlp/input_files/annual_reports"
vocabulary = get_keywords_from_file("/Users/david/Projects/fin-disclosures-nlp/data/keyword_vocabulary.txt")

def get_unlabelled_paragraphs_of_report(report_row, add_adjunct_pages = True):
    # Set tracking vars
    no_hot_neg_labels = 0
    no_weak_neg_labels = 0
    
    # Load report
    path = os.path.join(parsed_reports_folder,report_row['input_file'])
    folder = os.path.dirname(path)
    parsed_report_file_path = os.path.join(folder, report_row['orig_report_type'] + '_' + str(int(report_row['year'])), report_row['output_file'])
    
    # Get pages with keyword hits
    pages = get_counts_per_page(parsed_report_file_path, vocabulary)
    
    page_indizes = set(pages.index)
    
    # Add adjunct pages if necessary
    if add_adjunct_pages:
        for p in pages.index:
            if p > 0:
                page_indizes.add(p - 1)
            # elif p < TOTAL_PAGES:
            page_indizes.add(p + 1)
    
    # For each page, get all paragraphs
    for page_no in page_indizes:
        try:
            text = get_text_from_page(parsed_report_file_path, page_no)
        except IndexError:
            continue
        processed_doc = DocumentPreprocessor(text).process()
        paragraphs = processed_doc.split('\n\n')
        
        # Get the positive labels for this page
        pos_labels_paragraph_no = df_labels_positive.query("report_id == @report_row.id & page == @page_no").paragraph_no
        
        # Get the "hot" paragraphs, i.e. those that were containing a bigram of the search vocab
        hot_paragraphs = get_count_matrix(paragraphs, vocabulary)
        hot_paragraphs = hot_paragraphs.sum(axis=1)
        hot_paragraphs = np.squeeze(np.asarray(hot_paragraphs))
        hot_paragraphs = np.where(hot_paragraphs)[0]
        
        # The "weak" paragraphs are all those that are not either in hot or the actual positive labels
        weak_paragraphs = np.setdiff1d(np.arange(len(paragraphs)), hot_paragraphs, assume_unique=True)
        weak_paragraphs = np.setdiff1d(weak_paragraphs, pos_labels_paragraph_no)
        hot_paragraphs_filtered = np.setdiff1d(hot_paragraphs, pos_labels_paragraph_no)
        
        for p in weak_paragraphs:
            weak_negative_labels.append([report_row.id, page_no, p, paragraphs[p], "EXTRACTED", "weak"])
            no_weak_neg_labels += 1
            
        for p in hot_paragraphs_filtered:
            hot_negative_labels.append([report_row.id, page_no, p, paragraphs[p], "EXTRACTED", "hot"])
            no_hot_neg_labels += 1
    
    # Add summary stats for each row, i.e. how many "positive/negative" labels
    report_row["no_pos_labels"] = len(df_labels_positive.query("report_id == @report_row.id"))
    report_row["no_neg_hot_labels"] = no_hot_neg_labels
    report_row["no_neg_weak_labels"] = no_weak_neg_labels
    print(f"Done with {report_row.id}. Extracted {no_hot_neg_labels} (hot) and {no_weak_neg_labels} (weak) negative labels...")
    return report_row

labelled_reports.loc[0:].apply(lambda row: get_unlabelled_paragraphs_of_report(row), axis=1)

# Append to negative labels dataframe
df_hot_neg_labels = pd.DataFrame(hot_negative_labels, columns=["report_id", "page", "paragraph_no", "text", "coder", "neg_type"])
df_weak_neg_labels = pd.DataFrame(weak_negative_labels, columns=["report_id", "page", "paragraph_no", "text", "coder", "neg_type"])
df_labels_negative = df_labels_negative.append(df_hot_neg_labels, ignore_index=True)
df_labels_negative = df_labels_negative.append(df_weak_neg_labels, ignore_index=True)

Done with de_sap-AR_2001. Extracted 0 (hot) and 0 (weak) negative labels...
Done with nl_asml_hldg-AR_2000. Extracted 2 (hot) and 82 (weak) negative labels...
Done with de_sap-AR_2005. Extracted 7 (hot) and 168 (weak) negative labels...
Done with de_sap-AR_1999. Extracted 0 (hot) and 0 (weak) negative labels...
Done with de_sap-AR_2011. Extracted 49 (hot) and 584 (weak) negative labels...
Done with de_sap-AR_2018. Extracted 170 (hot) and 2456 (weak) negative labels...
Done with nl_asml_hldg-AR_2012. Extracted 0 (hot) and 24 (weak) negative labels...
Done with nl_asml_hldg-AR_2002. Extracted 0 (hot) and 43 (weak) negative labels...
Done with de_sap-AR_2009. Extracted 22 (hot) and 414 (weak) negative labels...
Done with nl_asml_hldg-AR_2013. Extracted 0 (hot) and 39 (weak) negative labels...
Done with gb_vodafone_grp-AR_2011. Extracted 11 (hot) and 176 (weak) negative labels...
Done with es_telefonica-AR_1999. Extracted 5 (hot) and 129 (weak) negative labels...
Done with de_deutsche_tele

IndexError: list index out of range

In [109]:
# Now filter out all "good" negative examples
df_labels_positive = df_labels_positive.query("cro.notnull()").copy()

## Overlapping labels correction

Remove overlapping labels (labels that are in both positive and negative due to a change in the paragraph parsing approach)

In [84]:
df_labels_overlapping

,report_id,neg_page,neg_paragraph_no,neg_text,neg_type,pos_paragraph_no,pos_text,pos_cro,pos_cro_sub_type,labelling_dataset
0,de_sap-AR_2009,134,8,\nTerrorist attacks and other acts of violence...,hot,7,"Social and political instability caused, for e...",PR,ACUTE,training
1,de_sap-AR_2009,138,1,"\nFor SAP, sustainability is a standard that g...",hot,0,SAP’s sustainability strategy may be difficult...,TR,REPUT,training
2,gb_vodafone_grp-AR_2011,48,12,\nWe are dependent on the secure operation of ...,hot,7,Our business could be adversely affected by di...,PR,ACUTE,training
3,es_telefonica-AR_2016,170,22,\nPower: Technology update for power units and...,hot,21,PsF (Power saving Features): Activation of ene...,OP,EFFI,training
4,es_telefonica-AR_2018,28,1,"\nScope 2: Scope 2 emissions, from power consu...",hot,0,Scope 1: Our Scope 1 emissions come from two m...,OP,ENERGY,training
...,...,...,...,...,...,...,...,...,...,...
101,de_basf-AR_2007,102,9,"O), ammonia and other\ninorganic compounds suc...",weak,7,"O), ammonia and other\ninorganic compounds suc...",OP,RESILI,training
102,de_basf-AR_2007,102,10,"\nIn 2007, emissions to air from BASF’s chemic...",weak,7,"O), ammonia and other\ninorganic compounds suc...",OP,RESILI,training
103,de_basf-AR_2007,103,7,"\nIn 2007, BASF’s water requirements worldwide...",weak,6,We had set ourselves the goal of reducing emis...,OP,EFFI,training
104,de_basf-AR_2012,41,2,\nThe number and quality of our patents attest...,weak,1,Our research pipeline included approximately 3...,OP,PRODUCTS,training


In [79]:
df_labels_positive = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_Positive.pkl")
df_labels_negative = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_Negative.pkl")
df_labels_overlapping = pd.read_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_Overlapping.pkl")

In [85]:
no_negative_before = len(df_labels_negative)
df_labels_negative


,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined,neg_type
0,nl_asml_hldg-AR_2013,NaN,NaN,44,1,True,"negative,neg co2 example",The�number�of�lithography�systems�we�are�able�...,DF,False,False,False,NaN,NaN,None,manual
1,fr_axa-AR_2012,NaN,NaN,387,3,True,interesting example that is not directly an OP,(see above sections). Indeed insurers are well...,DF,False,False,False,NaN,NaN,None,manual
2,fr_axa-AR_2017,NaN,NaN,385,4,True,not directly a CRO but interesting example on ...,AXA is reviewing a framework that models trans...,DF,False,False,False,NaN,NaN,None,manual
3,fr_axa-AR_2017,NaN,NaN,385,5,True,not directly a CRO but interesting example on ...,These results can be translated into a forward...,DF,False,False,False,NaN,NaN,None,manual
4,fr_axa-AR_2017,NaN,NaN,385,7,True,not directly a CRO but interesting example on ...,"In addition to the above “transition” risks, c...",DF,False,False,False,NaN,NaN,None,manual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26809,gb_national_grid-AR_2006,NaN,NaN,175,31,NaN,NaN,Personal Lifetime Allowance\nThe Lifetime Allo...,EXTRACTED,NaN,NaN,NaN,NaN,NaN,NaN,weak
26810,gb_national_grid-AR_2006,NaN,NaN,175,32,NaN,NaN,SEC\nUS Securities and Exchange Commission.,EXTRACTED,NaN,NaN,NaN,NaN,NaN,NaN,weak
26811,gb_national_grid-AR_2006,NaN,NaN,175,34,NaN,NaN,"TW\nTerawatt, 1012 watts.",EXTRACTED,NaN,NaN,NaN,NaN,NaN,NaN,weak
26812,gb_national_grid-AR_2006,NaN,NaN,175,35,NaN,NaN,TWh\nTerawatt hours.,EXTRACTED,NaN,NaN,NaN,NaN,NaN,NaN,weak


In [89]:
for idx, row in df_labels_overlapping.iterrows():
    neg_row = df_labels_negative.query(f"report_id == '{row.report_id}' and page == {row.neg_page} and paragraph_no == {row.neg_paragraph_no}")
    df_labels_negative.drop(neg_row.index, axis=0, inplace=True)

assert len(df_labels_overlapping) == no_negative_before - len(df_labels_negative), f"Overlappings ({len(df_labels_overlapping)}) not matching number of diff in negatives ({no_negative_before - len(df_labels_negative)})"

AssertionError: Overlappings (106) not matching number of diff in negatives (0)

In [110]:
df_labels_positive.to_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_Positive.pkl", protocol=4)
df_labels_negative.to_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training_Negative.pkl", protocol=4)

In [91]:
df_labels = pd.concat([df_labels_positive, df_labels_negative], ignore_index=True)
df_labels.groupby("neg_type", dropna=False).count()

,report_id,cro,cro_sub_type,page,paragraph_no,label,comment,text,coder,indirect,vague,past,keyword,span_id,cro_sub_type_combined
neg_type,,,,,,,,,,,,,,,
hot,3328,0,0,3328,3328,0,0,3328,3328,0,0,0,0,0,0
manual,15,0,0,15,15,11,13,15,15,15,15,15,0,0,0
weak,23365,0,0,23365,23365,0,0,23365,23365,0,0,0,0,0,0
NaN,1172,1172,1171,1172,1172,1122,467,1172,1172,1172,1172,1172,101,76,1169


In [92]:
df_labels.to_pickle("/Users/david/Nextcloud/Dokumente/Education/Uni Bern/Master Thesis/Analyzing Financial Climate Disclosures with NLP/Labelling/annual reports/Firm_AnnualReport_Labels_Training.pkl", protocol=4)